In [395]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pyjstat import pyjstat
import plotly.express as px

# Data Cleaning 
I start off by just testing out the eurostat dataset, then make functions for getting specific data for countries/types of crime and predicting the crime levels

In [396]:
def get_eurostat_data(article_title, *args):
    '''
    To Do:
        - Write *args
        - Write _url_builder() helper func.
    '''
    
    article_title = article_title.upper()
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/{article_title}?format=JSON&lang=EN'

    output = pyjstat.Dataset.read(url)
    df = output.write('dataframe')

    return df

In [397]:
df_crime = get_eurostat_data("crim_off_cat")

In [398]:
df_crime.loc[:, "Time"] = df_crime.loc[:, "Time"].astype(int)
df_crime

,Time frequency,International classification of crime for statistical purposes (ICCS),Unit of measure,Geopolitical entity (reporting),Time,value
0,Annual,Intentional homicide,Number,Belgium,2008,204.00
1,Annual,Intentional homicide,Number,Belgium,2009,189.00
2,Annual,Intentional homicide,Number,Belgium,2010,189.00
3,Annual,Intentional homicide,Number,Belgium,2011,214.00
4,Annual,Intentional homicide,Number,Belgium,2012,206.00
...,...,...,...,...,...,...
25825,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2018,0.11
25826,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2019,0.17
25827,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2020,0.28
25828,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Kosovo*,2021,NaN


In [399]:

df_crime = df_crime.rename(columns={"Geopolitical entity (reporting)" : "country"})


df_crime.loc[df_crime.country == "Türkiye", "country"] = "Turkey"
df_crime["International classification of crime for statistical purposes (ICCS)"].unique()


array(['Intentional homicide', 'Attempted intentional homicide',
       'Serious assault', 'Kidnapping', 'Sexual violence', 'Rape',
       'Sexual assault', 'Sexual exploitation', 'Child pornography',
       'Robbery', 'Burglary', 'Burglary of private residential premises',
       'Theft', 'Theft of a motorized vehicle or parts thereof',
       'Unlawful acts involving controlled drugs or precursors', 'Fraud',
       'Corruption', 'Bribery', 'Money laundering',
       'Acts against computer systems',
       'Participation in an organized criminal group'], dtype=object)

In [400]:
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")
eu_countries


['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden']

In [401]:
df_crime = df_crime[df_crime["country"].isin(eu_countries)]
df_homicide_be = df_crime[(df_crime.country == "Belgium") & (df_crime["Unit of measure"] == "Per hundred thousand inhabitants" ) & 
                                            (df_crime["International classification of crime for statistical purposes (ICCS)"] == "Intentional homicide")]





In [402]:
#get large dataset, clean it
df = get_eurostat_data("crim_off_cat")
df["Time"] = df["Time"].astype(int)

df = df.rename(columns={"Time" : "Year"})


#get EU countries
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")

#rename to country
df = df.rename(columns={"Geopolitical entity (reporting)" : "country", 
                        "value" : "Amount (Per 100k)"})

#change turkeys name
df.loc[df.country == "Türkiye", "country"] = "Turkey"

#include only EU countries
df = df[df["country"].isin(eu_countries)]


def get_crime_data(country, *args):
     if len(args) > 0:
          return df[(df.country == country) & (df["Unit of measure"] == "Per hundred thousand inhabitants" ) & 
                                            (df["International classification of crime for statistical purposes (ICCS)"] == args[0])]
     return df[(df.country == country) & (df["Unit of measure"] == "Per hundred thousand inhabitants" )]

    
    

In [403]:
df["International classification of crime for statistical purposes (ICCS)"].unique()

violent = ['Intentional homicide', 'Attempted intentional homicide',
       'Serious assault', 'Kidnapping']

sexual = ['Sexual violence', 'Rape',
       'Sexual assault', 'Sexual exploitation', 'Child pornography']

stealing = ['Robbery', 'Burglary', 'Burglary of private residential premises',
       'Theft', 'Theft of a motorized vehicle or parts thereof']

drugs = ['Unlawful acts involving controlled drugs or precursors']

white_collar = ['Fraud',
       'Corruption', 'Bribery', 'Money laundering',
       'Acts against computer systems',
       'Participation in an organized criminal group']

crime_categories = [violent, sexual, stealing, drugs, white_collar]

df_crime = get_eurostat_data("crim_off_cat")





In [404]:
crime_dict = {"country" : [], "violent" : [], "sexual" : [], "stealing" : [], "drugs" : [], "white_collar" : []}
#test = []


#so inefficient, but works (could optimize but already had get_crime_data method written)
for country in eu_countries:

    crime_dict["country"].append(country)
    i = 0
    for category in crime_categories:
       
       sum = 0;
       crime_bool = (df_crime["International classification of crime for statistical purposes (ICCS)"] == category[0])
       for crime in category[1:]:
            crime_bool = (crime_bool) | (df_crime["International classification of crime for statistical purposes (ICCS)"] == crime)
        
        
       crime_bool = (crime_bool) & (df.country == country) & (df["Unit of measure"] == "Per hundred thousand inhabitants" ) & (df_crime["Time"] == "2022")
       
       df_category_crime = df_crime[crime_bool]

       #if df_category_crime.shape != df_category_crime.dropna().shape :
           #test.append(country)
           #print(category[0])
           
       crime_dict[list(crime_dict.keys())[i + 1]].append(df_category_crime["value"].sum())
       i += 1

pd.DataFrame(crime_dict)

,country,violent,sexual,stealing,drugs,white_collar
0,Austria,50.47,131.76,1764.21,392.39,1074.51
1,Belgium,572.04,202.26,2664.35,484.04,1141.27
2,Bulgaria,48.77,17.53,575.33,78.32,45.17
3,Croatia,23.74,52.32,592.68,237.81,172.74
4,Cyprus,22.87,31.72,203.05,113.30,70.62
5,Czechia,41.03,43.48,754.10,39.94,203.97
6,Denmark,40.12,271.67,3468.77,532.38,1133.53
7,Estonia,6.39,49.48,656.56,235.02,235.62
8,Finland,38.94,201.32,2560.43,454.31,1096.18
9,France,583.80,270.45,2500.81,488.82,417.65


Now, I'm showing an individual data set's graph

In [405]:
crime_data = get_crime_data("Austria", 'Intentional homicide')
print(crime_data)
px.scatter(data_frame=crime_data, x="Year", y="Amount (Per 100k)")


    Time frequency  \
900         Annual   
901         Annual   
902         Annual   
903         Annual   
904         Annual   
905         Annual   
906         Annual   
907         Annual   
908         Annual   
909         Annual   
910         Annual   
911         Annual   
912         Annual   
913         Annual   
914         Annual   

    International classification of crime for statistical purposes (ICCS)  \
900                               Intentional homicide                      
901                               Intentional homicide                      
902                               Intentional homicide                      
903                               Intentional homicide                      
904                               Intentional homicide                      
905                               Intentional homicide                      
906                               Intentional homicide                      
907                            

Now, I'm going to add up all the crime per country, per year. Because each crime is in the per 100k units, i can simply add them up to get the total amount of crimes per 100k for a given country for a given year

In [406]:

df_crime_100k = df_crime[ df_crime["Unit of measure"] == "Per hundred thousand inhabitants" ]
df_crime_100k_nona = df_crime_100k.dropna()


country_lst = []
year_lst = []
amount = []
for country in eu_countries:
    df_country = get_crime_data(country)
    min = df_country["Year"].min()
    max = df_country["Year"].max()

    df_crime_country = pd.DataFrame()
    
    for year in range(min, max + 1):
        df_country_year = df_country[df_country["Year"] == year]
        sum = df_country_year["Amount (Per 100k)"].sum()
        country_lst.append(country)
        year_lst.append(year)
        amount.append(sum)
    

df_total_crime = pd.DataFrame({"country" : country_lst, "year" : year_lst, "amount" : amount})
df_total_crime.loc[df_total_crime.country == "Czechia", "country"] = "Czech Republic"

print(df_total_crime)


print(df_total_crime.corr()) 
px.scatter(df_total_crime, x="year", y="amount", color="country")

df_total_crime.to_csv("../Phase 3/Data/crime_training.csv", index=False)
df_total_crime = pd.read_csv("../Phase 3/Data/crime_training.csv")

     country  year   amount
0    Austria  2008  3896.74
1    Austria  2009  3906.74
2    Austria  2010  3308.21
3    Austria  2011  3197.13
4    Austria  2012  3212.19
..       ...   ...      ...
400   Sweden  2018  9620.21
401   Sweden  2019  9423.61
402   Sweden  2020  9299.21
403   Sweden  2021  8730.09
404   Sweden  2022  8482.97

[405 rows x 3 columns]
            year    amount
year    1.000000 -0.109631
amount -0.109631  1.000000


# Time to Scale it!!

In [407]:

mean = df_total_crime.describe()["amount"]["mean"]

std = df_total_crime.describe()["amount"]["std"]




df_total_crime_scaled = pd.DataFrame()

df_total_crime_scaled["country"] = df_total_crime["country"]
df_total_crime_scaled["year"] = df_total_crime["year"]



df_total_crime_scaled["amount"] = (df_total_crime["amount"] - df_total_crime["amount"].mean()) / df_total_crime["amount"].std()

df_total_crime_scaled.loc[df_total_crime_scaled.country == "Czechia", "country"] = "Czech Republic"



px.scatter(df_total_crime_scaled, x="year", y="amount", color="country")
print(mean)
print(std)

2631.533481481479
1944.3487156161827


In [415]:


df_total_crime_dummies = df_total_crime_scaled.join(pd.get_dummies(df_total_crime_scaled["country"]))
crime_arr = np.array(df_total_crime_dummies.drop(columns=["country"]))
crime_arr




array([[2.00800000e+03, 6.50709674e-01, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00900000e+03, 6.55852784e-01, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.01000000e+03, 3.48022200e-01, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.02000000e+03, 3.42925961e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.02100000e+03, 3.13655491e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.02200000e+03, 3.00945837e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [409]:
def add_bias_column(X):
    """
    Args:
        X (array): can be either 1-d or 2-d
    
    Returns:
        Xnew (array): the same array, but 2-d with a column of 1's in the first spot
    """
    
    # If the array is 1-d
    if len(X.shape) == 1:
        Xnew = np.column_stack([np.ones(X.shape[0]), X])
    
    # If the array is 2-d
    elif len(X.shape) == 2:
        bias_col = np.ones((X.shape[0], 1))
        Xnew = np.hstack([bias_col, X])
        
    else:
        raise ValueError("Input array must be either 1-d or 2-d")

    return Xnew

def line_of_best_fit(X, y):
    X = add_bias_column(X)
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))



In [417]:
X_left = crime_arr[:, 0:1]
X_right = crime_arr[:, 2:]

X = np.hstack((X_left, X_right))

y = crime_arr[:, 1:2]




#regression line for all crime data
regression = line_of_best_fit(X, y)

line_dict = {"intercept" : regression[0], "year" : regression[1]}
for index in range(len(eu_countries)):
    line_dict[eu_countries[index]] = regression[index + 2][0]
line_dict["mean"] = mean
line_dict["std"] = std


#test add
np.dot(add_bias_column(X)[0, :], regression)
pd.DataFrame(line_dict).to_csv("../Phase 3/Data/crime_inference.csv")



In [411]:
def lin_reg_crime (country, year):
    """
    following linear regression model, predicts the total amount of crimes per 100k people

    Args:
       - country (int) - which country wanted for prediction
       - year (int) - which year we wanted for prediction

    Returns:
       - answer (int) - predicted value for crimes per 100k people
    
    """
    countries = df_total_crime_dummies.keys().to_list()[3:]

    encoding = [0] * len(countries)

    encoding[countries.index(country)] = 1

    encoding.insert(0, year)
    encoding.insert(0, 1)

    scaled_answer = np.matmul(encoding, regression)[0]




    answer = scaled_answer * df_total_crime["amount"].std() + df_total_crime["amount"].mean()

    return answer



lin_reg_crime("Romania", 2025) 

794.1223532142872

# Leisure Activities

In [412]:
df_leisure = get_eurostat_data("ilc_scp01")
df_leisure["Age class"].unique()

array(['From 16 to 24 years', 'From 16 to 29 years',
       'From 16 to 44 years', 'From 16 to 64 years', '16 years or over',
       'From 20 to 64 years', 'From 25 to 29 years',
       'From 25 to 34 years', 'From 25 to 64 years',
       'From 30 to 54 years', 'From 35 to 44 years',
       'From 35 to 49 years', 'From 45 to 54 years',
       'From 45 to 64 years', 'From 50 to 64 years',
       'From 55 to 64 years', 'From 65 to 74 years', '65 years or over',
       'From 75 to 84 years', '75 years or over'], dtype=object)

In [413]:
df_leisure[(df_leisure["Age class"] == "16 years or over") & (df_leisure["Sex"] == "Total") & (df_leisure["Frequency"] == "At least once") 
           & (df_leisure["Time"] == "2022") & (df_leisure["Geopolitical entity (reporting)"].isin(eu_countries)) & (df_leisure["International Standard Classification of Education (ISCED 2011)"] == "All ISCED 2011 levels")]


,Time frequency,Frequency,International Standard Classification of Education (ISCED 2011),Age class,Sex,Unit of measure,Geopolitical entity (reporting),Time,value
27221,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Belgium,2022,64.3
27224,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Bulgaria,2022,21.3
27227,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Czechia,2022,62.6
27230,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Denmark,2022,80.7
27233,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Germany,2022,NaN
27236,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Estonia,2022,63.3
27239,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Ireland,2022,70.2
27242,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Greece,2022,40.0
27245,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,Spain,2022,63.3
27248,Annual,At least once,All ISCED 2011 levels,16 years or over,Total,Percentage,France,2022,59.1


In [414]:
age_classes = [(16, 24), (25, 34), (35, 44), (45, 54), (55, 64), (65, 74), 75]
df_leisure_subset = df_leisure[(df_leisure["Sex"] == "Total") & (df_leisure["Frequency"] == "At least once") 
           & (df_leisure["Time"] == "2022") & (df_leisure["Geopolitical entity (reporting)"].isin(eu_countries)) & (df_leisure["International Standard Classification of Education (ISCED 2011)"] == "All ISCED 2011 levels")]

leisure_by_age = pd.DataFrame()
leisure_dict = {"country" : df_leisure_subset["Geopolitical entity (reporting)"].unique().tolist()}

for ages in age_classes:
    if type(ages) is int:
        temp = df_leisure_subset[df_leisure_subset["Age class"] == f'{ages} years or over']["value"].tolist()
        leisure_dict[f'leisure_{ages}_older'] = temp
    else:
        temp = df_leisure_subset[df_leisure_subset["Age class"] == f'From {ages[0]} to {ages[1]} years']["value"].tolist()
        leisure_dict[f'leisure_{ages[0]}_{ages[1]}'] = temp
    

    
    
    

leisure_df = pd.DataFrame(leisure_dict)
leisure_df.loc[leisure_df.country == "Czechia", "country"] = "Czech Republic"
leisure_df


    

    
    

,country,leisure_16_24,leisure_25_34,leisure_35_44,leisure_45_54,leisure_55_64,leisure_65_74,leisure_75_older
0,Belgium,82.0,74.5,74.2,63.7,59.6,55.1,36.2
1,Bulgaria,42.6,31.0,28.4,21.7,15.0,10.0,4.5
2,Czech Republic,85.9,73.0,71.9,66.9,60.2,48.8,29.4
3,Denmark,91.5,86.6,82.9,84.1,77.4,74.3,64.8
4,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Estonia,86.9,79.5,75.4,70.1,56.8,45.9,26.2
6,Ireland,85.9,81.4,79.5,71.7,66.9,49.3,33.4
7,Greece,69.6,61.7,47.7,43.8,36.0,22.3,10.3
8,Spain,86.3,79.1,69.8,66.5,59.8,50.6,29.0
9,France,83.0,71.1,69.3,62.8,53.7,47.2,27.7


Ages classes: 16-24, 25-34, 35-44, 45-54, 55-64, 65-74, 75+

Units: percentage